<a href="https://colab.research.google.com/github/popo169/Portfolio-Project/blob/main/Copy_of_Pizza_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Business intelligence is very important nowadays. It helps you to know better of your company including how the sales recently, which item is going to out of stock and most importantly, what is the profit.

In this project, I am going to demonstrate how to use data from a pizza shop along with SQL to create 3 dashboards for business intelligence.

##Data content

There are 10 tables in the dataset.

Table name | Content | Columns
--- | --- | ---
Address | Customers' address and zip code | 'add_id', 'delivery_address1', 'delivery_address2', 'delivery_city', 'delivery_zipcode'
Customers | First and last name | 'cust_id', 'cust_firstname', 'cust_lastname'
Ingredient | Ingredient name, weight, unit and price | 'ing_id', 'ing_name', 'ing_weight', 'ing_meas', 'ing_price'
Inventory | Quantity of inventory | 'inv_id', 'item_id', 'quantity'
Item | Item name, code, category, size and price | 'item_id', 'sku', 'item_name', 'item_cat', 'item_size', 'item_price'
Orders | Order create time, item ordered, quantity, customer ID, delivery/pick up, address ID | 'row_id', 'order_id', 'created_at', 'item_id', 'quantity', 'cust_id', 'delivery', 'add_id'
Recipe | Recipe ID, ingredient ID, required quantity | 'row_id', 'recipe_id', 'ing_id', 'quantity'
Rota | Duty date, shift ID, staff ID | 'row_id', 'rota_id', 'date', 'shift_id', 'staff_id'
Shift | Day of week, start time and end time | 'shift_id', 'day_of_week', 'start_time', 'end_time'
Staff | Staff first and last name, position, hourly rate | 'staff_id', 'first_name', 'last_name', 'position', 'hourly_rate'

##Dashboard 1

This dashboard contains the basic information about the pizza shop. It tells whether the pizza shop is earning or losing money.
1.   Total number of order
2.   Total sales
3.   Total item sold
4.   Average order value
5.   Sales by category
6.   Top selling item
7.   Orders by hour
8.   Sales by hour
9.   Orders by address
10.  Orders by delivery/pick up

##Dashboard 2

The second dashboard is about the material cost and stock information to keep the pizza shop running.
1.   Total quantity required for orders by ingredient
2.   Total cost of ingredients for orders
3.   Calculated cost of pizza
4.   Percentage stock remaining by ingredient
5.   List of ingredients to re-order based on remaining inventory

##Dashboard 3

The last dashboard is going to capture the human resource cost of the pizza shop.
1.   Hours worked by staff member
2.   Total hours worked
3.   Cost per staff member
4.   Total staff cost

##Import data

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import library
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
# import data
address=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/address.csv')
customers=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/customers.csv')
ingredient=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/ingredient.csv')
inventory=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/inventory.csv')
item=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/item.csv')
orders=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/orders.csv')
recipe=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/recipe.csv')
rota=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/rota.csv')
shift=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/shift.csv')
staff=pd.read_csv('/content/gdrive/My Drive/dataset/SQL/pizza_proj/staff.csv')

In [ ]:
# import data to sql tables
pd_to_sqlDB(address,"address","pizza_db")
pd_to_sqlDB(customers,"customers","pizza_db")
pd_to_sqlDB(ingredient,"ingredient","pizza_db")
pd_to_sqlDB(inventory,"inventory","pizza_db")
pd_to_sqlDB(item,"item","pizza_db")
pd_to_sqlDB(orders,"orders","pizza_db")
pd_to_sqlDB(recipe,"recipe","pizza_db")
pd_to_sqlDB(rota,"rota","pizza_db")
pd_to_sqlDB(shift,"shift","pizza_db")
pd_to_sqlDB(staff,"staff","pizza_db")